# Basics

## A Very Simple Example

A simple fortran code consists of a _main program_ containing a series of instructions. Modern versions typically have a file extension `.f90`, which signifies free format source. The `.f` extension indicates old-style fixed format.
The use of the `!` symbol tells the compiler than any subsequent text on that line is a _comment_. Comments can have a line to themselves or they can follow code and it's always a good idea to use comments to make sure your code is well documented. **However**, don't hide code within comments; keep the two easily distinguishable to make the source easy to read.

In [ ]:
%num_images: 1
Program hello              ! The name of the program is a matter of personal preference. 

  ! The Implict none command tells the compiler to treat undeclared variables as errors.
  ! It's strongly advised to used this as it catches spelling mistakes and forces the
  ! programmer to give each variable a type.
  Implicit none
  
  Integer     :: myNumber  ! Declare the variable myNumber to be of type 'integer'
  
  myNumber = 25            ! Assign a value to a variable
  
  Print *, 'Hello World!'  ! Comments can also follow Fortran instructions
  Print *, 'I like the number', myNumber

End program                ! The program must terminate with this line.

### Compiling the Code

The above code has to be compiled before it can be run, converting it from (near) English to machine language. Using the _GNU Fortran_ compiler and assuming the file containing the source was named `hello.f90`, this could be achieved as follows:

```bash
gfortran -o hello hello.f90
```

This will produce an executable file called `hello`, which can be executed on Linux:

```bash
./hello
```

### Code Description

The above example, while basic, demonstrates some of the fundamental concepts that are applicable to any Fortran program:

 * Units or sections (in this case the _program_).
 * Variables, used to hold information.
  * Type.
  * Declaration.
  * Assignment.
  * De-referencing.
 * Output

Each of these topics will be discussed in further detail, but will be familiar to those with experience in other programming languages.

### Exercise 1 - _Hello World!_

Copy the above code into a text file named `ex1.f90`, using a text editor such as `nano`.
 * Compile and run it.
 * Change the output produced by the program. Does it still run?